In [10]:
!pip install -q langchain langchain-google-genai "google-ai-generativelanguage>=0.6.18,<0.7.0"

In [24]:
import os
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda, RunnableMap
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser


In [30]:
# 환경 변수에서 키 불러오기
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID", "")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET", "")

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)
parser = StrOutputParser()

# 🟠 1단계: 캠핑 준비물 생성
def generate_checklist(state):
    user_input = state["question"]
    prompt = f"""
    사용자가 '{user_input}' 라고 했어. 캠핑에 필요한 준비물 목록을 5~7개 정도 추천해줘.
    단순 나열 형식으로 출력해줘 (예: 텐트, 침낭, 휴대용 가스버너, ...)"""
    checklist = llm.invoke([HumanMessage(content=prompt)])
    print(f"\n📝 [generate_checklist] question: {state['question']}")
    return {"checklist": checklist.content.strip()}

In [26]:

# 🟠 2단계: 각 준비물에 대해 SEO 최적 키워드 생성
def generate_keywords(state):
    checklist = state["checklist"].split(", ")
    keyword_prompt = f"""
    다음 캠핑 준비물 리스트가 있어: {', '.join(checklist)}.
    각각에 대해 네이버 쇼핑 검색에 적합한 SEO 최적 키워드 (상세하고 일반적으로 검색되는 형태)로 바꿔줘.
    예시 형식:
    텐트 → 4인용 자동 캠핑 텐트
    침낭 → 겨울용 거위털 침낭
    형식 유지해서 출력해줘.
    """
    keyword_response = llm.invoke([HumanMessage(content=keyword_prompt)])
    lines = keyword_response.content.strip().split("\n")
    keywords = [line.split("→")[1].strip() for line in lines if "→" in line]
    return {"keywords": keywords}

In [27]:

# 🟠 3단계: 네이버 API로 검색
def search_naver_items(state):
    results = []
    for keyword in state["keywords"]:
        url = "https://openapi.naver.com/v1/search/shop.json"
        headers = {
            "X-Naver-Client-Id": NAVER_CLIENT_ID,
            "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
        }
        params = {"query": keyword, "display": 3, "start": 1, "sort": "sim"}
        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:
            items = response.json().get("items", [])
            results.append({
                "keyword": keyword,
                "items": items
            })
    return {"search_results": results}


In [28]:

# 🟠 4단계: 상품 비교 및 추천 (LLM 활용)
def compare_and_recommend(state):
    recommendation = []
    for group in state["search_results"]:
        keyword = group["keyword"]
        items = group["items"]
        summary_prompt = f"""
        다음은 '{keyword}'로 검색한 네이버 쇼핑 상품 3개야:

        {items[0]['title']} - {items[0]['lprice']}원
        {items[1]['title']} - {items[1]['lprice']}원
        {items[2]['title']} - {items[2]['lprice']}원

        각각의 장단점을 간단히 비교해주고, 그 중 가장 추천하는 하나의 제품을 골라줘.
        """
        summary = llm.invoke([HumanMessage(content=summary_prompt)])
        recommendation.append({
            "keyword": keyword,
            "summary": summary.content.strip()
        })
    return {"recommendations": recommendation}

In [31]:
from typing import TypedDict, List, Dict
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

# 상태 스키마 정의
class AppState(TypedDict):
    question: str
    checklist: str
    keywords: List[str]
    search_results: List[Dict]
    recommendations: List[Dict]

# ✅ 올바른 LangGraph 구성
builder = StateGraph(state_schema=AppState)
builder.add_node("ChecklistGenerator", RunnableLambda(generate_checklist))
builder.add_node("KeywordGenerator", RunnableLambda(generate_keywords))
builder.add_node("NaverSearcher", RunnableLambda(search_naver_items))
builder.add_node("Recommender", RunnableLambda(compare_and_recommend))

builder.set_entry_point("ChecklistGenerator")
builder.add_edge("ChecklistGenerator", "KeywordGenerator")
builder.add_edge("KeywordGenerator", "NaverSearcher")
builder.add_edge("NaverSearcher", "Recommender")
builder.add_edge("Recommender", END)

app = builder.compile()

# 실행
output = app.invoke({"question": "나는 내일 캠핑갈건데 뭐가 필요할까?"})
for rec in output["recommendations"]:
    print(f"\n📌 [{rec['keyword']}] 추천 요약:\n{rec['summary']}")


IndexError: list index out of range

In [32]:
# -*- coding: utf-8 -*-
!pip install -q langchain langchain-google-genai "google-ai-generativelanguage>=0.6.18,<0.7.0"

import os
import requests
from typing import TypedDict, List, Dict
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage

# 🔐 환경 변수 설정 또는 기본값
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY", "")
NAVER_CLIENT_ID = os.getenv("NAVER_CLIENT_ID", "")
NAVER_CLIENT_SECRET = os.getenv("NAVER_CLIENT_SECRET", "")

# 🔗 LLM 초기화
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)

# 🟠 1단계: 캠핑 준비물 생성
def generate_checklist(state):
    user_input = state["question"]
    prompt = f"""
    사용자가 '{user_input}' 라고 했어. 캠핑에 필요한 준비물 목록을 5~7개 정도 추천해줘.
    단순 나열 형식으로 출력해줘 (예: 텐트, 침낭, 휴대용 가스버너, ...)"""
    checklist = llm.invoke([HumanMessage(content=prompt)]).content.strip()

    print("\n📝 [ChecklistGenerator]")
    print("📥 입력 질문:", user_input)
    print("📤 준비물 리스트:", checklist)

    return {"question": user_input, "checklist": checklist}

# 🟠 2단계: SEO 최적 키워드 생성
def generate_keywords(state):
    checklist = state["checklist"].split(", ")
    prompt = f"""
    다음 캠핑 준비물 리스트가 있어: {', '.join(checklist)}.
    각각에 대해 네이버 쇼핑 검색에 적합한 SEO 최적 키워드 (상세하고 일반적으로 검색되는 형태)로 바꿔줘.
    예시 형식:
    텐트 → 4인용 자동 캠핑 텐트
    침낭 → 겨울용 거위털 침낭
    형식 유지해서 출력해줘.
    """
    response = llm.invoke([HumanMessage(content=prompt)]).content.strip()
    lines = response.split("\n")
    keywords = [line.split("→")[1].strip() for line in lines if "→" in line]

    print("\n🔑 [KeywordGenerator]")
    print("📥 입력 체크리스트:", checklist)
    print("📤 추출된 키워드:", keywords)

    return {"checklist": state["checklist"], "keywords": keywords}

# 🟠 3단계: 네이버 쇼핑 API 검색
def search_naver_items(state):
    results = []
    for keyword in state["keywords"]:
        url = "https://openapi.naver.com/v1/search/shop.json"
        headers = {
            "X-Naver-Client-Id": NAVER_CLIENT_ID,
            "X-Naver-Client-Secret": NAVER_CLIENT_SECRET
        }
        params = {"query": keyword, "display": 3, "start": 1, "sort": "sim"}
        response = requests.get(url, headers=headers, params=params)
        items = response.json().get("items", []) if response.status_code == 200 else []

        print(f"\n🔍 [NaverSearcher] 키워드: '{keyword}'")
        print(f"🔎 검색된 상품 수: {len(items)}")

        results.append({"keyword": keyword, "items": items})

    return {"keywords": state["keywords"], "search_results": results}

# 🟠 4단계: 상품 비교 및 추천
def compare_and_recommend(state):
    recommendations = []
    for group in state["search_results"]:
        keyword = group["keyword"]
        items = group["items"]

        print(f"\n🤖 [Recommender] 키워드: '{keyword}'")
        print(f"🛒 상품 수: {len(items)}")

        if len(items) < 3:
            summary = f"'{keyword}'에 대한 상품이 부족하여 비교할 수 없습니다."
        else:
            summary_prompt = f"""
            다음은 '{keyword}'로 검색한 네이버 쇼핑 상품 3개야:

            {items[0]['title']} - {items[0]['lprice']}원
            {items[1]['title']} - {items[1]['lprice']}원
            {items[2]['title']} - {items[2]['lprice']}원

            각각의 장단점을 간단히 비교해주고, 그 중 가장 추천하는 하나의 제품을 골라줘.
            """
            summary = llm.invoke([HumanMessage(content=summary_prompt)]).content.strip()

        recommendations.append({
            "keyword": keyword,
            "summary": summary
        })

    return {"search_results": state["search_results"], "recommendations": recommendations}

# 💡 상태 스키마 정의
class AppState(TypedDict):
    question: str
    checklist: str
    keywords: List[str]
    search_results: List[Dict]
    recommendations: List[Dict]

# 🧠 LangGraph 구성
builder = StateGraph(state_schema=AppState)
builder.add_node("ChecklistGenerator", RunnableLambda(generate_checklist))
builder.add_node("KeywordGenerator", RunnableLambda(generate_keywords))
builder.add_node("NaverSearcher", RunnableLambda(search_naver_items))
builder.add_node("Recommender", RunnableLambda(compare_and_recommend))

builder.set_entry_point("ChecklistGenerator")
builder.add_edge("ChecklistGenerator", "KeywordGenerator")
builder.add_edge("KeywordGenerator", "NaverSearcher")
builder.add_edge("NaverSearcher", "Recommender")
builder.add_edge("Recommender", END)

app = builder.compile()

# 🚀 실행
output = app.invoke({"question": "나는 내일 캠핑갈건데 뭐가 필요할까?"})

# ✅ 최종 결과 출력
print("\n🎯 최종 추천 요약 결과")
for rec in output["recommendations"]:
    print(f"\n📌 [{rec['keyword']}] 추천 요약:\n{rec['summary']}")



📝 [ChecklistGenerator]
📥 입력 질문: 나는 내일 캠핑갈건데 뭐가 필요할까?
📤 준비물 리스트: 텐트, 침낭, 매트, 휴대용 가스버너, 코펠, 의자, 헤드랜턴

🔑 [KeywordGenerator]
📥 입력 체크리스트: ['텐트', '침낭', '매트', '휴대용 가스버너', '코펠', '의자', '헤드랜턴']
📤 추출된 키워드: ['4인용 돔텐트 / 가족캠핑텐트 / 자동텐트 설치 쉬운 텐트 / 초보자 캠핑텐트 / 2인용 캠핑텐트 / 캠핑 텐트 추천', '겨울용 침낭 / 여름용 침낭 / 봄가을용 침낭 / 거위털 침낭 / 오리털 침낭 / 침낭 추천 / 캠핑 침낭', '캠핑 매트 / 자동차 캠핑 매트 / 에어 매트 / 발포 매트 / 캠핑 매트리스 / 두꺼운 캠핑 매트 /  침낭 매트 세트', '휴대용 가스 버너 / 부탄 가스 버너 / 캠핑 가스 버너 / 휴대용 버너 추천 / 미니 가스 버너 /  캠핑 요리용 가스버너', '캠핑 코펠 세트 / 2인용 코펠 / 4인용 코펠 / 티타늄 코펠 / 스텐레스 코펠 / 코펠 추천 / 접이식 코펠', '캠핑 의자 / 접이식 캠핑 의자 / 캠핑 체어 / 휴대용 캠핑 의자 / 목베개 캠핑 의자 / 릴렉스 체어 / 캠핑 의자 추천', '캠핑 헤드랜턴 / LED 헤드랜턴 / 방수 헤드랜턴 / 충전식 헤드랜턴 /  다기능 헤드랜턴 /  밝은 헤드랜턴 / 헤드랜턴 추천']

🔍 [NaverSearcher] 키워드: '4인용 돔텐트 / 가족캠핑텐트 / 자동텐트 설치 쉬운 텐트 / 초보자 캠핑텐트 / 2인용 캠핑텐트 / 캠핑 텐트 추천'
🔎 검색된 상품 수: 3

🔍 [NaverSearcher] 키워드: '겨울용 침낭 / 여름용 침낭 / 봄가을용 침낭 / 거위털 침낭 / 오리털 침낭 / 침낭 추천 / 캠핑 침낭'
🔎 검색된 상품 수: 3

🔍 [NaverSearcher] 키워드: '캠핑 매트 / 자동차 캠핑 매트 / 에어 매트 / 발포 매트 / 캠핑 매트리스 / 두꺼운 캠핑 매트 /  침낭 매트 세트'
🔎 검색된 상품 수: 